In [4]:
import pandas as pd
fedrate = pd.read_csv("../.Database/fedrate.csv")
pd.set_option('display.max_columns', None)

In [8]:
wanted_columns = ['Year','Month','Effective Federal Funds Rate']
fedrate = fedrate[wanted_columns]


In [10]:
fedrate

,Year,Month,Effective Federal Funds Rate
0,1954,7,0.80
1,1954,8,1.22
2,1954,9,1.06
3,1954,10,0.85
4,1954,11,0.83
...,...,...,...
899,2016,12,NaN
900,2017,1,0.65
901,2017,2,0.66
902,2017,3,NaN


In [15]:
loan = pd.read_csv("../.Database/cleaned_loan.csv")
loan['issue_d'] = loan['issue_d'] + '-01'  # Adding a default day of 1
loan['issue_d'] = pd.to_datetime(loan['issue_d'], format='%b-%y-%d')
loan['issue_y'] = loan['issue_d'].dt.year
loan['issue_m'] = loan['issue_d'].dt.month

ValueError: time data "Dec-2011-01" doesn't match format "%b-%y-%d", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [18]:
merged_df = pd.merge(loan, fedrate, left_on=['issue_y', 'issue_m'], right_on=['Year', 'Month'], how='left')
merged_df

,annual_inc,delinq_2yrs,dti,emp_title,emp_length,funded_amnt,grade,home_ownership,inq_last_6mths,installment,int_rate,issue_d,loan_amnt,loan_status,mths_since_last_delinq,pub_rec,purpose,sub_grade,term,title,total_acc,total_pymnt,acc_now_delinq,log_annual_inc,loan_status_grouped,issue_y,issue_m,Year,Month,Effective Federal Funds Rate
0,24000.0,0.0,27.65,NaN,10+ years,5000,B,RENT,1.0,162.87,10.65,2011-12-01,5000,Fully Paid,0.0,0.0,credit_card,B2,36 months,Computer,9.0,5861.071414,0.0,10.085809,0.0,2011,12,2011,12,0.07
1,30000.0,0.0,1.00,Ryder,< 1 year,2500,C,RENT,5.0,59.83,15.27,2011-12-01,2500,Charged Off,0.0,0.0,car,C4,60 months,bike,4.0,1008.710000,0.0,10.308953,1.0,2011,12,2011,12,0.07
2,12252.0,0.0,8.72,NaN,10+ years,2400,C,RENT,2.0,84.33,15.96,2011-12-01,2400,Fully Paid,0.0,0.0,small_business,C5,36 months,real estate business,10.0,3003.653644,0.0,9.413444,0.0,2011,12,2011,12,0.07
3,49200.0,0.0,20.00,AIR RESOURCES BOARD,10+ years,10000,C,RENT,1.0,339.31,13.49,2011-12-01,10000,Fully Paid,35.0,0.0,other,C1,36 months,personel,37.0,12226.302210,0.0,10.803649,0.0,2011,12,2011,12,0.07
4,36000.0,0.0,11.20,Veolia Transportaton,3 years,5000,A,RENT,3.0,156.46,7.90,2011-12-01,5000,Fully Paid,0.0,0.0,wedding,A4,36 months,My wedding loan I promise to pay back,12.0,5631.377753,0.0,10.491274,0.0,2011,12,2011,12,0.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260213,63078.0,0.0,31.70,customer service,10+ years,17000,D,MORTGAGE,0.0,413.32,15.99,2015-01-01,17000,Fully Paid,0.0,0.0,debt_consolidation,D2,60 months,Debt consolidation,28.0,17196.520000,0.0,11.052127,0.0,2015,1,2015,1,0.11
260214,48000.0,0.0,36.93,supervisor,10+ years,4200,D,MORTGAGE,0.0,147.64,15.99,2015-01-01,4200,Charged Off,38.0,0.0,medical,D2,36 months,Medical expenses,45.0,1026.020000,0.0,10.778956,1.0,2015,1,2015,1,0.11
260215,54000.0,1.0,13.22,Coordinator of RSVP,< 1 year,10775,A,RENT,0.0,327.95,6.03,2015-01-01,10775,Fully Paid,16.0,0.0,debt_consolidation,A1,36 months,Debt consolidation,21.0,11071.870000,0.0,10.896739,0.0,2015,1,2015,1,0.11
260216,27000.0,0.0,18.58,Painter,2 years,6225,D,RENT,1.0,220.37,16.49,2015-01-01,6225,Fully Paid,0.0,0.0,debt_consolidation,D3,36 months,Debt consolidation,4.0,7050.460000,0.0,10.203592,0.0,2015,1,2015,1,0.11


In [1]:
import pandas as pd
unemp = pd.read_csv("../.Database/us_state_unemployment_rate.csv")
pd.set_option('display.max_columns', None)

In [2]:
unemp

,state_code,year,month,value
0,AL,2015,12,6.1
1,AL,2015,11,6.1
2,AL,2015,10,6.1
3,AL,2015,9,6.1
4,AL,2015,8,6.2
...,...,...,...,...
5395,WY,2007,5,2.7
5396,WY,2007,4,2.6
5397,WY,2007,3,2.6
5398,WY,2007,2,2.6


In [3]:
loan = pd.read_csv("../.Database/cleaned_loan.csv")
loan

,annual_inc,addr_state,delinq_2yrs,dti,emp_title,emp_length,funded_amnt,grade,home_ownership,inq_last_6mths,installment,int_rate,issue_d,loan_amnt,loan_status,mths_since_last_delinq,pub_rec,purpose,sub_grade,term,title,total_acc,total_pymnt,acc_now_delinq,log_annual_inc,loan_status_grouped
0,24000.0,AZ,0.0,27.65,NaN,10+ years,5000.0,B,RENT,1.0,162.87,10.65,Dec-2011,5000.0,Fully Paid,0.0,0.0,credit_card,B2,36 months,Computer,9.0,5861.071414,0.0,10.085809,0.0
1,30000.0,GA,0.0,1.00,Ryder,< 1 year,2500.0,C,RENT,5.0,59.83,15.27,Dec-2011,2500.0,Charged Off,0.0,0.0,car,C4,60 months,bike,4.0,1008.710000,0.0,10.308953,1.0
2,12252.0,IL,0.0,8.72,NaN,10+ years,2400.0,C,RENT,2.0,84.33,15.96,Dec-2011,2400.0,Fully Paid,0.0,0.0,small_business,C5,36 months,real estate business,10.0,3003.653644,0.0,9.413444,0.0
3,49200.0,CA,0.0,20.00,AIR RESOURCES BOARD,10+ years,10000.0,C,RENT,1.0,339.31,13.49,Dec-2011,10000.0,Fully Paid,35.0,0.0,other,C1,36 months,personel,37.0,12226.302212,0.0,10.803649,0.0
4,36000.0,AZ,0.0,11.20,Veolia Transportaton,3 years,5000.0,A,RENT,3.0,156.46,7.90,Dec-2011,5000.0,Fully Paid,0.0,0.0,wedding,A4,36 months,My wedding loan I promise to pay back,12.0,5631.377753,0.0,10.491274,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257772,63078.0,PA,0.0,31.70,customer service,10+ years,17000.0,D,MORTGAGE,0.0,413.32,15.99,Jan-2015,17000.0,Fully Paid,0.0,0.0,debt_consolidation,D2,60 months,Debt consolidation,28.0,17196.520000,0.0,11.052127,0.0
257773,48000.0,CO,0.0,36.93,supervisor,10+ years,4200.0,D,MORTGAGE,0.0,147.64,15.99,Jan-2015,4200.0,Charged Off,38.0,0.0,medical,D2,36 months,Medical expenses,45.0,1026.020000,0.0,10.778956,1.0
257774,54000.0,FL,1.0,13.22,Coordinator of RSVP,< 1 year,10775.0,A,RENT,0.0,327.95,6.03,Jan-2015,10775.0,Fully Paid,16.0,0.0,debt_consolidation,A1,36 months,Debt consolidation,21.0,11071.870000,0.0,10.896739,0.0
257775,27000.0,FL,0.0,18.58,Painter,2 years,6225.0,D,RENT,1.0,220.37,16.49,Jan-2015,6225.0,Fully Paid,0.0,0.0,debt_consolidation,D3,36 months,Debt consolidation,4.0,7050.460000,0.0,10.203592,0.0


In [ ]:
merged_df = pd.merge(loan, unemp, left_on=['issue_y', 'issue_m'], right_on=['Year', 'Month'], how='left')
merged_df